<a href="https://colab.research.google.com/github/IndranilPaul99/Read_alignment/blob/main/Bioinformatics_2022_CSE_C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Creating Test Data**

## Function to generate a random Reference Genome

In [1]:
import random

def generate_Random_Reference_Genome( length_of_genome ) :
    bases = ["A" , "C" , "G" , "T"]
    genome = open("reference_genome","w")
    for i in range( length_of_genome ) :
        genome.write(random.choice(bases))
    genome.close()
    print(" Generation Complete!!")

In [41]:
#@title Generate Reference Genome { vertical-output: true, display-mode: "form" }
length_of_genome = 10000 #@param {type:"integer"}

generate_Random_Reference_Genome( length_of_genome )


 Generation Complete!!


## Function to create a Sample Genome from the Reference Genome with dissimilarities

In [ ]:
import random

def create_Sample_Genome( variation_percentage ) :
    bases = {
        "A" : ["C" , "G" , "T" ],
        "C" : ["A" , "G" , "T" ],
        "G" : ["A" , "C" , "T" ],
        "T" : ["A" , "C" , "T" ]
    }

    reference_genome = open("reference_genome","r")
    sample_genome = open("sample_genome","w")

    count = 0
    while True:
        base = reference_genome.read(1)
        if not base :
            break
        r = random.uniform(0,100)
        if( r <= variation_percentage ) :
            count += 1
            sample_genome.write( random.choice(bases[base]) )
        else :
            sample_genome.write( base )
    reference_genome.close()
    sample_genome.close()
    print( " Number of dissimilarities = " , count )

In [ ]:
#@title Generate Sample Genome { vertical-output: true, display-mode: "form" }
variance_of_sample_genome = 2 #@param {type:"slider", min:0, max:20, step:0.1}

create_Sample_Genome( variance_of_sample_genome )

## Function to create reads from Sample Genome
Offset of each read is saved for later verification.

In [ ]:
import random

def generate_Reads( min_read_length , max_read_length , iterations ) :
    sample_reads = open( "sample_reads" , "w" )
    read_indexes = open( "reads_indexes" , "w" )
    sample_genome = open( "sample_genome" , "r" )

    count = 0
    for _ in range(iterations) :
        sample_genome.seek(0)
        while True :
            position_of_read = sample_genome.tell()
            read_length = random.randint( min_read_length , max_read_length )
            read = sample_genome.read( read_length )
            if not read :
                break
            sample_reads.writelines( read + "\n" )
            read_indexes.write( str(position_of_read) + "\n" )
            count += 1
    
    sample_reads.close()
    read_indexes.close()
    sample_genome.close()
    print( " Number of reads produced = " , count )

In [ ]:
#@title Extract Reads from Sample Genome { vertical-output: true, display-mode: "form" }
min_read_length =  20#@param {type:"integer"}
max_read_length = 30 #@param {type:"integer"}
iterations =  15#@param {type:"number"}

if min_read_length < max_read_length and min_read_length < length_of_genome :
    generate_Reads( min_read_length , max_read_length , iterations )
else :
    print( " Invalid values !!! " )


# **Producing the Burrows-Wheller Transform**

## Making the Burrows-Wheeler Transform
Offset are also sorted in the order of the rotations i.e. the suffix array is created. This array records the position of each base of the BWT in the reference genome.

In [50]:
def burrows_wheeler() :
    genome = open( "reference_genome" , "r" ).read() + "$"
    offsets = [ i for i in range( len( genome ) ) ]
    offsets.sort( key = lambda i : genome[i:] + genome[:i] )

    with open( "l_col" , "w" ) as file1 , open( "r_col" , "w" ) as file2 , open( "offsets" , "w" ) as file3 :
        for i in offsets :
            rotation = genome[i:] + genome[:i]
            file1.write( rotation[0] )
            file2.write( rotation[-1] )
            file3.write( str(i) + "\n" )

    print( "BWT Produced !!!" )

burrows_wheeler()

BWT Produced !!!


## Making the Rank , Skip and Select Tables

In [51]:
def generate_Rank_Select_Skip() :
    bwt = open( "r_col" , "r" ).read()
    rank = { "$" : [] , "A" : [] , "C" : [] , "G" : [] , "T" : [] }
    temp_ranks = { "$" : -1 , "A" : -1 , "C" : -1 , "G" : -1 , "T" : -1 }
    skip = { "$" : 0 , "A" : 0 , "C" : 0 , "G" : 0 , "T" : 0 }
    select = { "$" : [] , "A" : [] , "C" : [] , "G" : [] , "T" : [] }
    
    for base in bwt :
        temp_ranks[base] += 1
        for ch in [ "$" , "A" , "C" , "G" , "T" ] :
            rank[ch].append( temp_ranks[ch] )
    prev = "$"
    for curr in [ "A" , "C" , "G" , "T" ] :
        skip[curr] = temp_ranks[prev] + skip[prev] + 1
        prev = curr
    for i in range( len( bwt ) ) :
        select[ bwt[i] ].append( i )
    with open( "rank" , "w" ) as file1 , open( "skip" , "w" ) as file2 , open( "select" , "w" ) as file3 :
        for ch in [ "$" , "A" , "C" , "G" , "T" ] :
            for num in rank[ch] :
                file1.write( str( num ) + " " )
            file1.write( "\n" )
            file2.write( str( skip[ch] ) + "\n" )
            for num in select[ch] :
                file3.write( str( num ) + " " )
            file3.write( "\n" )
    print( "rank , select , skip generated!!!" )

generate_Rank_Select_Skip()

rank , select , skip generated!!!


## Reversing The Burrows-Wheller Transform

In [52]:
def reverse_BWT() :
    rank = {}
    skip = {}
    with open( "rank" , "r" ) as f1 , open( "skip" , "r" ) as f2 :
        for base in [ "$" , "A" , "C" , "G" , "T" ] :
            rank.update( { base : [ int(i) for i in f1.readline().strip( " \n" ).split( " " ) ] } )
            skip.update( { base : int( f2.readline() ) } )
    bwt = open( "r_col" , "r" ).read()
    
    reversed_bwt = ""
    offset = 0
    while True :
        base = bwt[ offset ]
        if base is "$" :
            break
        reversed_bwt = base + reversed_bwt
        offset = skip[ base ] + rank[ base ][ offset ]
    
    reference_genome = open( "reference_genome" , "r" ).read()
    if reversed_bwt == reference_genome :
        print( "BWT reversal successful !!!" )
    else :
        print( "BWT reversal unsuccessful." )

reverse_BWT()

BWT reversal successful !!!
